In [1]:
#how to create tool using @tool decorator
#simplest way to define custom tool,decorator uses function name as the toool name by default,
#but this can be overridden by passing string as the first argument, 
#decorator will use the function's docstring as tool's description - so docstring must provided

from langchain_core.tools import tool

@tool
def divison(a: int,b: int)->int:
    """Divide 2 numbers"""
    return a/b

In [2]:
print(divison.name)
print(divison.description)
print(divison.args)

divison
Divide 2 numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [3]:
#async implementation
from langchain_core.tools import tool

@tool
async def multiply(a: int,b: int)->int:
    """Multiplies 2 numbers"""
    return a*b

In [4]:
from typing import Annotated,List

@tool
def multiply_by_max(
    a:Annotated[int,"A value"],
    b:Annotated[List[int],"B value"]
)->int:
    """Multiply a by the maximum of b"""
    return a*max(b)

In [5]:
print(multiply_by_max.args)

{'a': {'description': 'A value', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'B value', 'items': {'type': 'integer'}, 'title': 'B', 'type': 'array'}}


In [8]:
#Structured tool
#it gives a bit more configurability than the @tool decorator, without requiring much additional code
from langchain_core.tools import StructuredTool

def mutliply(a:int,b:int)->int:
    """Mutliply 2 numbers"""
    return a*b

async def amutliply(a:int,b:int)->int:
    """Mutliply 2 numbers asynchronusly"""
    return a*b

calculator=StructuredTool.from_function(func=mutliply,coroutine=amutliply)
#in coroutine gives async function

print(calculator.invoke({"a":3,"b":2}))
print(await calculator.ainvoke({"a":5,"b":4}))


6
20


Langchain inbuilt tools
https://docs.langchain.com/oss/javascript/integrations/tools

https://docs.langchain.com/oss/python/integrations/retrievers

In [4]:
#inbuilt tools

#wikipedia

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=500)

wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper)

print(wiki_tool.invoke({"query":"langchain"}))




Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.




In [2]:
from langchain_community.utilities import ArxivAPIWrapper

tool=ArxivAPIWrapper()

tool.run("1706.03762")

'Published: 2023-08-02\nTitle: Attention Is All You Need\nAuthors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin\nSummary: The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new 

In [5]:
from langchain_core.tools import tool

@tool
def add(a:int,b:int)->int:
    """Adds 2 numbers"""
    return a+b

@tool
def multiply(a:int,b:int)->int:
    """multiplys 2 numbers"""
    return a*b

In [6]:
#call tool with llm model
tools=[wiki_tool,add,multiply]

tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Vansh Parmar\\OneDrive\\Documents\\Generative_AI\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=500)),
 StructuredTool(name='add', description='Adds 2 numbers', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x000002512E9A9580>),
 StructuredTool(name='multiply', description='multiplys 2 numbers', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x000002512E9A8D60>)]

In [7]:
from langchain.chat_models import init_chat_model

llm=init_chat_model("qwen/qwen3-32b",model_provider="groq")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002512D97EC30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002512ED2C6B0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
llm_with_tools=llm.bind_tools(tools)
llm_with_tools

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002512D97EC30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002512ED2C6B0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'wikipedia', 'description': 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'query to look up on wikipedia', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}, {'type': 'function

In [14]:
from langchain_core.messages import HumanMessage
query="What is 2*3"
messages=[HumanMessage(query)]
response=llm_with_tools.invoke(query)
print(response)

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking "What is 2*3". Let me see. I need to figure out which tool to use here. The available tools are Wikipedia, add, and multiply. The question is a multiplication problem, so the multiply function should handle it. The parameters required for multiply are a and b, both integers. Here, a is 2 and b is 3. So I should call the multiply function with these values. No need to use Wikipedia since it\'s a straightforward math problem. Let me make sure there\'s no trick here. 2 multiplied by 3 is 6. Yep, that\'s simple. So the correct tool call is multiply with arguments 2 and 3.\n', 'tool_calls': [{'id': '3w9gn58mj', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 174, 'prompt_tokens': 324, 'total_tokens': 498, 'completion_time': 0.287131159, 'completion_tokens_details': {'reasoning_tokens': 145}, 'prompt_time': 0.014189325, 'pr

In [15]:
response.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': '3w9gn58mj',
  'type': 'tool_call'}]

In [16]:

for tool_call in response.tool_calls:
    selected_tool={"add":add,"multiply":multiply,"wikipedia":wiki_tool}[tool_call["name"].lower()]
    tool_msg=selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 2*3', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='6', name='multiply', tool_call_id='3w9gn58mj')]

In [17]:
llm_with_tools.invoke(messages)

AIMessage(content='The result of multiplying 2 and 3 is **6**.', additional_kwargs={'reasoning_content': 'Okay, the user asked "What is 2*3". I need to figure out the right tool to use here. Looking at the available functions, there\'s a multiply function that takes two integers. The parameters are a and b, both integers. So, 2 and 3 are the numbers to multiply. I should call the multiply function with a=2 and b=3. The result is 6, which is the answer the user is looking for. Let me make sure there\'s no confusion with the add function. No, the user specifically used the multiplication symbol (*), so multiply is correct. Alright, the tool call was made correctly, and the response was 6. That\'s all there is to it.\n'}, response_metadata={'token_usage': {'completion_tokens': 167, 'prompt_tokens': 334, 'total_tokens': 501, 'completion_time': 0.312368142, 'completion_tokens_details': {'reasoning_tokens': 148}, 'prompt_time': 0.016349966, 'prompt_tokens_details': None, 'queue_time': 0.0570

In [18]:
from langchain_core.messages import HumanMessage

query="What is langchain and what id 15*15"
messages=[HumanMessage(query)]

ai_msg=llm_with_tools.invoke(messages)
ai_msg

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking two things here. First, they want to know what LangChain is. Second, they\'re asking for the product of 15 multiplied by 15. Let me break this down.\n\nFor the first part, "What is LangChain?" I need to use the Wikipedia tool to get an accurate description. LangChain is likely a term related to technology or programming, maybe a framework or library. Using the Wikipedia function will help fetch the latest and most reliable information about it.\n\nThen, the second part is a math problem: 15 times 15. The user provided a function called "multiply" which takes two integers. I should call this function with a=15 and b=15 to get the result. \n\nWait, but the user also mentioned "id 15*15". Maybe "id" here is a typo or shorthand. Since the main query is about multiplying 15 by 15, I\'ll proceed with the multiply function. \n\nSo, I need to make two separate tool calls. One to Wikipedia for LangChain and 

In [20]:
ai_msg.tool_calls

[{'name': 'wikipedia',
  'args': {'query': 'LangChain'},
  'id': 'ed60dh88m',
  'type': 'tool_call'},
 {'name': 'multiply',
  'args': {'a': 15, 'b': 15},
  'id': 'aqvbtx2x5',
  'type': 'tool_call'}]

In [21]:
for tool_call in ai_msg.tool_calls:
    selected_tool={"add":add,"multiply":multiply,"wikipedia":wiki_tool}[tool_call["name"].lower()]
    tool_msg=selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is langchain and what id 15*15', additional_kwargs={}, response_metadata={}),
 ToolMessage(content="Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n", name='wikipedia', tool_call_id='ed60dh88m'),
 ToolMessage(content='225', name='multiply', tool_call_id='aqvbtx2x5')]

In [22]:
res=llm_with_tools.invoke(messages)
res.content

'LangChain is a software framework that helps integrate large language models (LLMs) into applications, with use cases like document analysis, chatbots, and code analysis.\n\nThe result of 15 multiplied by 15 is 225.'